# ex_network_A


Design a 1D neural network of 1000 cells receiving a 1D thalamic inputs at 50, 55, 60, 65, 70 ms.

The thalamo-cx connectivity is one-to-one.

Model of cells is free of choice.

Record and plot spikes, voltage and both the conductances.

In [19]:
# fileName

fileName = 'ex_network_A'

In [20]:
# libs

import pyNN.spiNNaker as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt


In [21]:
# simulation settings

sim.setup(
        timestep=1, # [ms]
        min_delay=1, # [ms]
        max_delay=100) # [ms]

simtime = 250 # [ms+]


2021-04-18 18:18:22 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-18 18:18:22 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-18 18:18:22 WARNING: /home/spinnaker/CNS-tutorial-2021/SpiNNaker/reports has 40 old reports that have not been closed
2021-04-18 18:18:22 WARNING: /home/spinnaker/CNS-tutorial-2021/SpiNNaker/application_generated_data_files has 41 old reports that have not been closed
2021-04-18 18:18:22 INFO: Setting time scale factor to 1.
2021-04-18 18:18:22 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


In [22]:
# make the network

structure = {}
structure['1D'] = space.Line(
                              dx=1.0, 
                              x0=0.0, 
                              y=0.0, 
                              z=0.0),
sizePop = 1000
cx = {}
cx['tot'] = sim.Population(
                    sizePop,
                    sim.IF_cond_exp,
                    cellparams=sim.IF_cond_exp.default_parameters,
                    structure = structure['1D'],
                    initial_values={'v': sim.RandomDistribution('uniform', (-70.0, -60.0))},
                    label='cx'
                   )



In [23]:
# set recording of state variables

cx['tot'].record(['spikes','v','gsyn_exc','gsyn_inh'])

In [24]:
# generate first ten position of cells

structure.generate_positions(10)

AttributeError: 'dict' object has no attribute 'generate_positions'

In [25]:
# make the thalamic input sources

thalamus = {}
thalamus['A'] = sim.Population(
                                1000,
                                sim.SpikeSourceArray(spike_times=[50, 55, 60, 65, 70]),
                                cellparams=None,
                                structure=structure['1D'],
                                initial_values=None,
                                label=None,
                                constraints=None,
                                additional_parameters=None,
                                )


In [26]:
# make the projection

In [27]:
projs = {}

projs['thal', 'cx'] = sim.Projection(
                                    thalamus['A'],
                                    cx['tot'] ,
                                    sim.OneToOneConnector(),
                                    synapse_type=sim.StaticSynapse(weight=0.1, delay=1),
                                    source=None,
                                    receptor_type='excitatory',
                                    space = space.Space(axes = 'x'),
                                    label='thal->cx-e',
                                )



In [28]:
# run simulation

sim.run(simtime) 

2021-04-18 18:18:22 INFO: Simulating for 250 1.0ms timesteps using a hardware timestep of 1000us
2021-04-18 18:18:22 INFO: Starting execution process
2021-04-18 18:18:26 INFO: Time 0:00:03.523345 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-18 18:18:35 INFO: Time 0:00:09.294031 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-18 18:18:40 INFO: Time 0:00:05.057134 taken by PartitionAndPlacePartitioner
Created spalloc job 5958234
2021-04-18 18:18:40 INFO: Created spalloc job 5958234
Waiting for board power commands to complete.
2021-04-18 18:18:40 INFO: Waiting for board power commands to complete.
2021-04-18 18:18:45 INFO: Time 0:00:05.047402 taken by SpallocAllocator
2021-04-18 1

250.0

In [29]:
# save the results


outputs = {}

outputs['cx'] = cx['tot'].get_data()
for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
    cx['tot'].write_data(fileName + '_' + str(recording) + '.pkl')


AttributeError: 'tuple' object has no attribute 'describe'

In [ ]:
# recover results function

def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results


In [ ]:
# recover results

results = recover_results(outputs)
results.keys()

In [ ]:
# check the spikes

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes
idx = 0
axes_list[idx].eventplot(results['cx', 'spikes'])
axes_list[idx].set_title('rasterplot of cx layer')
axes_list[idx].set_xlabel('[ms]')
axes_list[idx].set_ylabel('cells')
axes_list[idx].set_xlim(0, simtime)


In [ ]:
# check the voltage

fig, axes = plt.subplots(1, 1)#, figsize=(9,5))
fig.tight_layout(pad=4)
axes_list = fig.axes
idx = 0
im = axes_list[idx].imshow(results['cx', 'v'].T)
axes_list[idx].set_title('voltage of cx')
axes_list[idx].set_xlabel('time [ms]')
axes_list[idx].set_ylabel('cells')
fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')


In [ ]:
# check the conductances


for layer in ['cx']:
    fig, axes = plt.subplots(1, 2)
    fig.tight_layout(pad=4)
    fig.suptitle(str(layer) + ' layer')
    axes_list = fig.axes
    
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):
        im = axes_list[idx].imshow(results[layer, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

In [ ]:
# end simulation

sim.end()